In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'


In [2]:
r = requests.get(url)
json = r.json()
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [3]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
events_df = pd.DataFrame(json["events"])


In [4]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [5]:
slim_elements_df = elements_df[['second_name','first_name','team','element_type','selected_by_percent',
       'minutes','value_season','total_points','bps', 
       'influence', 'creativity', 'threat','goals_scored','assists','clean_sheets', 'goals_conceded', 
       'own_goals','penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards','saves', 'bonus']]

In [6]:
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

<ipython-input-6-d2f8756adc31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)


In [7]:
slim_elements_df.head()

,second_name,first_name,team,element_type,selected_by_percent,minutes,value_season,total_points,bps,influence,...,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,position
0,Leno,Bernd,1,1,2.3,3131,26.2,131,625,702.2,...,11,37,1,1,0,0,1,86,11,Goalkeeper
1,Rúnarsson,Rúnar Alex,1,1,1.2,15,0.2,1,8,16.6,...,0,0,0,0,0,0,0,2,0,Goalkeeper
2,Borges Da Silva,Willian,1,3,0.3,1398,12.0,78,306,256.2,...,3,17,0,0,0,0,0,0,9,Midfielder
3,Aubameyang,Pierre-Emerick,1,4,3.7,2330,13.1,131,358,493.8,...,10,26,1,0,0,2,0,0,11,Forward
4,Soares,Cédric,1,2,0.3,744,6.2,28,125,110.8,...,2,11,0,0,0,1,0,0,3,Defender


In [8]:
slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)
slim_elements_df['value_season'] = slim_elements_df.value_season.astype(float)

<ipython-input-8-853d5c83eb1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)
<ipython-input-8-853d5c83eb1e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df['value_season'] = slim_elements_df.value_season.astype(float)


In [9]:
slim_elements_df = slim_elements_df.sort_values('value_season', ascending = False)
slim_elements_df.head(10)

,second_name,first_name,team,element_type,selected_by_percent,minutes,value_season,total_points,bps,influence,...,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,position
32,Martínez,Emiliano,Aston Villa,1,39.0,3420,33.8,186,825,1067.2,...,15,46,0,1,0,1,0,142,27,Goalkeeper
227,Dallas,Stuart,Leeds,3,17.0,3410,31.1,171,726,756.8,...,12,53,0,0,0,7,0,0,15,Midfielder
238,Meslier,Illan,Leeds,1,5.1,3150,30.8,154,774,1026.0,...,11,52,1,1,0,0,0,140,20,Goalkeeper
451,Cresswell,Aaron,West Ham,2,19.2,3170,27.8,153,730,669.4,...,11,41,0,0,0,3,0,0,21,Defender
40,Targett,Matt,Aston Villa,2,9.7,3404,27.6,138,710,690.4,...,16,44,0,0,0,7,0,0,9,Defender
154,Guaita,Vicente,Crystal Palace,1,5.9,3330,27.6,124,639,819.4,...,8,64,0,1,0,2,0,107,15,Goalkeeper
385,Lloris,Hugo,Spurs,1,4.3,3420,27.1,149,702,868.0,...,12,45,0,0,0,0,0,114,13,Goalkeeper
239,Harrison,Jack,Leeds,3,5.5,2846,26.7,160,542,759.2,...,11,45,0,0,0,2,0,0,15,Midfielder
279,Santana de Moraes,Ederson,Man City,1,10.0,3240,26.7,160,696,586.0,...,19,28,0,1,0,3,0,66,3,Goalkeeper
448,Fabianski,Lukasz,West Ham,1,2.5,3150,26.6,133,634,745.2,...,10,44,0,1,0,2,0,102,11,Goalkeeper


In [13]:
slim_elements_df = slim_elements_df.loc[slim_elements_df["minutes"] >= 1]
try:
    slim_elements_df["Avg Playtime"] = slim_elements_df['minutes'] / 38
except:
    0
try:
    slim_elements_df["PPM"] = slim_elements_df["total_points"] / slim_elements_df["minutes"]
except:
    0
slim_elements_df.corr()

,element_type,minutes,value_season,total_points,bps,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,Avg Playtime,PPM
element_type,1.000000,-0.070792,-0.137307,0.083406,-0.141852,0.484958,0.384160,-0.053439,-0.052277,-0.158673,-0.342732,0.181868,0.021415,-0.023829,-0.473934,0.116591,-0.070792,0.069748
minutes,-0.070792,1.000000,0.933046,0.861753,0.937043,0.348094,0.415885,0.877351,0.907373,0.168161,0.211534,0.131968,0.504357,0.184475,0.281246,0.606597,1.000000,-0.138160
value_season,-0.137307,0.933046,1.000000,0.892486,0.944951,0.349306,0.390344,0.867791,0.805836,0.169115,0.283205,0.125669,0.364545,0.169186,0.366851,0.685905,0.933046,-0.087723
total_points,0.083406,0.861753,0.892486,1.000000,0.917642,0.676324,0.657229,0.845181,0.702563,0.089817,0.197368,0.242248,0.259655,0.108643,0.240780,0.853680,0.861753,-0.011578
bps,-0.141852,0.937043,0.944951,0.917642,1.000000,0.406212,0.470029,0.883024,0.788623,0.152740,0.250472,0.183605,0.392244,0.155138,0.323388,0.747856,0.937043,-0.075978
goals_scored,0.484958,0.348094,0.349306,0.676324,0.406212,1.000000,0.630380,0.346622,0.278138,-0.049121,-0.109949,0.349796,0.012737,0.039767,-0.155095,0.709411,0.348094,0.100650
assists,0.384160,0.415885,0.390344,0.657229,0.470029,0.630380,1.000000,0.423870,0.325768,-0.076457,-0.130448,0.250120,0.103325,-0.032715,-0.185115,0.617003,0.415885,0.057501
clean_sheets,-0.053439,0.877351,0.867791,0.845181,0.883024,0.346622,0.423870,1.000000,0.640450,0.130529,0.188751,0.193813,0.427390,0.177072,0.216231,0.591422,0.877351,-0.073249
goals_conceded,-0.052277,0.907373,0.805836,0.702563,0.788623,0.278138,0.325768,0.640450,1.000000,0.136001,0.179992,0.079069,0.490723,0.170479,0.268375,0.486323,0.907373,-0.165981
own_goals,-0.158673,0.168161,0.169115,0.089817,0.152740,-0.049121,-0.076457,0.130529,0.136001,1.000000,0.056359,-0.065202,0.070237,0.147081,0.063401,0.071081,0.168161,-0.048747


In [10]:
# slim_elements_df.to_csv('Player Data.csv')